In [1]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from pydantic import BaseModel,Field
from langchain_core.tools import StructuredTool,tool,ToolException

from langchain.agents import (AgentExecutor, create_tool_calling_agent)
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        ("system","You are a helpful assistant."),
        ("placeholder","{chat_history}"),
        ("human","{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)

In [5]:
@tool
def search(query:str) -> str:
    """
    Look up things online
    """
    return "Langchain"

In [6]:
print(search.name)
print(search.description)
print(search.args)

search
Look up things online
{'query': {'title': 'Query', 'type': 'string'}}


In [7]:
@tool
def multiply(a:int, b:int) -> int:
    """
    Multiply two numbers
    """
    return a*b

In [8]:
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
False


In [9]:
class SearchInput(BaseModel):
    query:str=Field(description="should be a search query")

@tool("search-tool",args_schema=SearchInput,return_direct=True)
def search(query:str) -> str:
    """
    Look up things online
    """
    return "Langchain"

In [10]:
print(search.name)
print(search.description)
print(search.args)
print(search.return_direct)

search-tool
Look up things online
{'query': {'description': 'should be a search query', 'title': 'Query', 'type': 'string'}}
True


<h3> StructuredTool DataClass <h3>

In [11]:
def searchFunction(query:str):
    return "Langchain"

search=StructuredTool.from_function(
    func=searchFunction,
    name="Search",
    description="Useful when you need to answer questions about current events"
)

In [12]:
print(search.name)
print(search.description)
print(search.args)

Search
Useful when you need to answer questions about current events
{'query': {'title': 'Query', 'type': 'string'}}


In [13]:
def exponentiate(base:float,exponent:float) -> float:
    """
        Exponentiate one number with another and return the output
        base: The Base Number
        exponent: The times you would exponentiate with
    """
    return base**exponent

exponent=StructuredTool.from_function(
    func=exponentiate,
    name="exponent",
    description="Useful when you need to find the exponent of any given base number"
)

In [14]:
tools=[exponent]

In [15]:
agent=create_tool_calling_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
# multiInputAgent=create_tool_calling_agent(llm=llmOpenAI.bind_tools(tools=[exponentiate]),tools=multiInputTools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [16]:
agentExecutor.invoke(input={"input":"How much is 52 to the power 12"})



> Entering new AgentExecutor chain...

Invoking: `exponent` with `{'base': 52, 'exponent': 12}`


3.908770064862502e+2052 to the power of 12 is approximately 3.91 x 10^20.

> Finished chain.


{'input': 'How much is 52 to the power 12',
 'output': '52 to the power of 12 is approximately 3.91 x 10^20.'}

<h3> Using ArgSchema with StructuredTool <h3>

In [4]:
class Calculator(BaseModel):
    a:int=Field(description="first number",le=5)
    b:int=Field(description="second number",le=5)

def multiply(a:int, b:int) -> int:
    """
        Multiply two numbers
    """
    return a*b

In [5]:
calculator=StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="Multiply Numbers",
    args_schema=Calculator,
    return_direct=True
)

In [6]:
print(calculator.name)
print(calculator.description)
print(calculator.args)
print(calculator.return_direct)

Calculator
Multiply Numbers
{'a': {'description': 'first number', 'maximum': 5, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'maximum': 5, 'title': 'B', 'type': 'integer'}}
True


In [7]:
calculator.invoke(input={'a':7,'b':6})

ValidationError: 2 validation errors for Calculator
a
  Input should be less than or equal to 5 [type=less_than_equal, input_value=7, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/less_than_equal
b
  Input should be less than or equal to 5 [type=less_than_equal, input_value=6, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/less_than_equal

In [8]:
calculator.invoke(input={'a':4,'b':3})

12

<h3> Handling Tool Errors </h3>

In [11]:
class Calculator(BaseModel):
    a:int=Field(description="first number")
    b:int=Field(description="second number")

def multiply(a:int, b:int) -> int:
    """
        Multiply two numbers less than 5
    """
    if a<5 and b<5:
        return a*b
    else:
        raise ToolException("Value of a and b should be less than 5")

calculator=StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="Multiply Numbers",
    args_schema=Calculator,
    return_direct=True,
    handle_tool_error=True  # This is a must for getting the error output
)

In [12]:
# calculator.invoke(input={'a':'a','b':6})
calculator.invoke(input={'a':6,'b':9})

'Value of a and b should be less than 5'

In [27]:
def search_tool1(s: str):
    try:
        return s/2
    except:
        raise ToolException("You cannot do this operation with a String")

In [28]:
search = StructuredTool.from_function(
    func=search_tool1,
    name="Search_tool1",
    description="A bad tool",
    handle_tool_error=True  # This is a must for getting the error output
)

search.invoke(input='65')

'You cannot do this operation with a String'